In [30]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from imblearn.over_sampling import SMOTE

In [31]:
df=pd.read_csv("creditcard.csv")

In [32]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [33]:
df['Class'].sum()

np.int64(492)

In [34]:
X = df.drop('Class', axis=1)
y = df['Class']

In [35]:
scaler = StandardScaler()
X[['Time', 'Amount']] = scaler.fit_transform(X[['Time', 'Amount']])

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [37]:
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

C:\Users\oshah\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
history = model.fit(X_train, y_train, epochs=20, batch_size=2048, validation_split=0.2, verbose=2)

Epoch 1/20
90/90 - 3s - 34ms/step - accuracy: 0.9883 - loss: 0.0970 - val_accuracy: 0.9982 - val_loss: 0.0130
Epoch 2/20
90/90 - 1s - 11ms/step - accuracy: 0.9983 - loss: 0.0174 - val_accuracy: 0.9987 - val_loss: 0.0053
Epoch 3/20
90/90 - 1s - 10ms/step - accuracy: 0.9987 - loss: 0.0098 - val_accuracy: 0.9992 - val_loss: 0.0044
Epoch 4/20
90/90 - 1s - 14ms/step - accuracy: 0.9988 - loss: 0.0075 - val_accuracy: 0.9993 - val_loss: 0.0041
Epoch 5/20
90/90 - 1s - 9ms/step - accuracy: 0.9990 - loss: 0.0065 - val_accuracy: 0.9993 - val_loss: 0.0040
Epoch 6/20
90/90 - 1s - 8ms/step - accuracy: 0.9990 - loss: 0.0063 - val_accuracy: 0.9993 - val_loss: 0.0039
Epoch 7/20
90/90 - 1s - 8ms/step - accuracy: 0.9991 - loss: 0.0053 - val_accuracy: 0.9993 - val_loss: 0.0038
Epoch 8/20
90/90 - 1s - 9ms/step - accuracy: 0.9991 - loss: 0.0049 - val_accuracy: 0.9993 - val_loss: 0.0038
Epoch 9/20
90/90 - 1s - 12ms/step - accuracy: 0.9992 - loss: 0.0051 - val_accuracy: 0.9993 - val_loss: 0.0038
Epoch 10/20
90

In [41]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


In [42]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.82      0.81      0.81        98

    accuracy                           1.00     56962
   macro avg       0.91      0.90      0.91     56962
weighted avg       1.00      1.00      1.00     56962



In [43]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[56847    17]
 [   19    79]]


In [44]:
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred_prob))

ROC-AUC Score: 0.9799937983071676


In [ ]:
!pip install keras-tuner
from kerastuner.tuners import RandomSearch

#model building function for tuner
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32),
                    activation='relu', input_dim=X_train_res.shape[1]))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.2, max_value=0.6, step=0.1)))
    if hp.Boolean('second_layer'):
        model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=128, step=32),
                        activation='relu'))
        model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.2, max_value=0.6, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

#Setup tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=2,
    directory='tuner_dir',
    project_name='credit_card_fraud')



   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   -------------------- ------------------- 1/2 [keras-tuner]
   ---------------------------------------- 2/2 [keras-tuner]



C:\Users\oshah\AppData\Local\Temp\ipykernel_37032\1034296094.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch
C:\Users\oshah\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
tuner.search(X_train_res, y_train_res, epochs=20, validation_split=0.2, batch_size=2048, verbose=2)
est_model = tuner.get_best_models(num_models=1)[0]

Trial 10 Complete [00h 01m 16s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 12m 19s


C:\Users\oshah\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\oshah\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [50]:
loss, accuracy = est_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9964 - loss: 0.0157
Test Accuracy: 0.9964


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Precision: 0.8229
Recall: 0.8061
F1-score: 0.8144
